<a href="https://colab.research.google.com/github/siddharth1704/Twitter_sentiment/blob/main/finalsentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os

In [5]:
import numpy as np
from numpy import array

In [43]:
from google.colab import files 
  
  
uploaded = files.upload()

Saving file2.csv to file2 (1).csv


In [44]:

import pandas as pd 
import io 
  
df = pd.read_csv(io.BytesIO(uploaded['file2.csv'])) 
print(df)


     Unnamed: 0                                             Tweets  sentiment
0             0  RT @AlistairHaimes: Of the whole omnishambles,...          1
1             1  RT @SteveBakerHW: The case for a new coronavir...          0
2             2  RT @atrupar: "If we tested half, cases would b...          1
3             3  @Jim_Jordan Wear a mask! 224,000+ American sou...          0
4             4  RT @BNODesk: The El Paso-Juárez area on the U....          0
..          ...                                                ...        ...
194         194  RT @DonaldJTrumpJr: Muslim Group Says Cuomo Is...          0
195         195  RT @HKLiberty_Team: #HKSARG relaxes max partic...          1
196         196  RT @AmyMcGrathKY: Sen. McConnell blocked $1,20...          1
197         197  RT @SteveBakerHW: The case for a new coronavir...          1
198         198  RT @nytimes: More than 75,000 coronavirus case...          1

[199 rows x 3 columns]


In [27]:
import re
#cleaning the data
def cleanTxt(text):
    text=re.sub(r'@[A-Za-z0-9,.'']+','',text) #remove @mentions
    text=re.sub(r'#','',text) #Removing the # symbol
    text=re.sub(r'https?:\/\/\S+','',text)#remove hyperlink
    text=re.sub(r'RT[\s]+','',text)#removing RT
    
    return text
#cleaning the new data
df['Tweets']=df['Tweets'].apply(cleanTxt)

In [28]:

import spacy
nlp = spacy.load('en_core_web_sm')
from spacy.lang.en.stop_words import STOP_WORDS
import string


In [29]:
stop_words = list(STOP_WORDS)
pun = string.punctuation

In [30]:
#removing the stopwords
def data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
        
    clean_tokens = []
    for token in tokens:
        if token not in stop_words and token not in pun:
            clean_tokens.append(token)
    return clean_tokens
# Function to convert 
def listToString(s): 
    # initialize an empty string 
    str1 = " "        # return string 
    return (str1.join(s))

In [31]:
ctweets = []
tweets = list(df['Tweets'])
for Tweet in tweets:
    ctweets.append(listToString(data_cleaning(Tweet)))

In [32]:
ctweets[3]

'jordan wear mask 224,000 american soul lose covid19 true patriot wear mask th …'

In [36]:
sentiments = df['sentiment']

from numpy import array
sentiments = np.array(list(sentiments))

In [38]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
ctweets_train, ctweets_test, sentiments_train, sentiments_test = train_test_split(ctweets, sentiments, test_size = 0.20, random_state = 42)

In [39]:

tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(ctweets_train)

ctweets_train = tokenizer.texts_to_sequences(ctweets_train)
ctweets_test = tokenizer.texts_to_sequences(ctweets_test)

In [40]:

vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

ctweets_train = pad_sequences(ctweets_train, padding='post', maxlen = maxlen)
ctweets_test = pad_sequences(ctweets_test, padding='post', maxlen = maxlen)


In [45]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-10-28 10:00:36--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-10-28 10:00:37--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-10-28 10:00:37--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [46]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [47]:
!ls
!pwd

'file2 (1).csv'   glove.6B.100d.txt   glove.6B.300d.txt   glove.6B.zip
 file2.csv	  glove.6B.200d.txt   glove.6B.50d.txt	  sample_data
/content


In [48]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dict = dict()
glove = open('glove.6B.100d.txt', encoding="utf8")

for line in glove:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:],dtype = 'float32')
    embeddings_dict [word] = vector_dimensions
glove.close()

In [49]:
embedding_matrix = zeros((vocab_size,100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [50]:

model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights = [embedding_matrix], input_length = maxlen, trainable = False)
model.add(embedding_layer)

model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))

In [51]:

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          67800     
_________________________________________________________________
flatten (Flatten)            (None, 10000)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 10001     
Total params: 77,801
Trainable params: 10,001
Non-trainable params: 67,800
_________________________________________________________________
None


In [55]:
history = model.fit(ctweets_train, sentiments_train, batch_size = 180, epochs = 40, verbose = 1, validation_split = 0.2)

Epoch 1/40
1/1 [==============================] - 0s 47ms/step - loss: 0.1753 - acc: 0.9843 - val_loss: 0.5419 - val_acc: 0.7188
Epoch 2/40
1/1 [==============================] - 0s 18ms/step - loss: 0.1707 - acc: 0.9843 - val_loss: 0.5425 - val_acc: 0.7188
Epoch 3/40
1/1 [==============================] - 0s 18ms/step - loss: 0.1663 - acc: 0.9843 - val_loss: 0.5433 - val_acc: 0.7188
Epoch 4/40
1/1 [==============================] - 0s 18ms/step - loss: 0.1622 - acc: 0.9843 - val_loss: 0.5440 - val_acc: 0.7188
Epoch 5/40
1/1 [==============================] - 0s 17ms/step - loss: 0.1582 - acc: 0.9843 - val_loss: 0.5449 - val_acc: 0.7188
Epoch 6/40
1/1 [==============================] - 0s 17ms/step - loss: 0.1545 - acc: 0.9843 - val_loss: 0.5459 - val_acc: 0.7188
Epoch 7/40
1/1 [==============================] - 0s 17ms/step - loss: 0.1509 - acc: 0.9843 - val_loss: 0.5469 - val_acc: 0.7188
Epoch 8/40
1/1 [==============================] - 0s 16ms/step - loss: 0.1474 - acc: 0.9843 - val

In [58]:
score  = model.evaluate(ctweets_test, sentiments_test, verbose = 1)

2/2 [==============================] - 0s 2ms/step - loss: 0.4633 - acc: 0.7500
